## Race, Class, and Student Debt as a Stratification System

### Research team: Charlie Eaton, Adam Goldstein, Laura Hamilton, Frederick Wherry

### Enrollment segregation IPEDS data exploration. Code by Charlie Eaton

### August 5, 2020

In [1]:
quietly {
foreach year in 2018 {
copy https://nces.ed.gov/ipeds/datacenter/data/EF`year'A_Data_Stata.zip EF`year'A_Data_Stata.zip, replace
unzipfile EF`year'A_Data_Stata, replace
insheet using ef`year'a_data_stata.csv, clear
quietly egen efminort = rowtotal (ef2mort efaiant efnhpit efasiat), missing
foreach var in totlt totlm totlw aiant aianm aianw asiat asiam ///
asiaw bkaat bkaam bkaaw hispt hispm hispw whitt whitm whitw ///
unknt unknm unknw 2mort 2morm 2morw nralt nralm nralw nhpit ///
nhpim nhpiw minort {
quietly gen ftfirstef`var'=ef`var' if line == 1
quietly gen ptfirstef`var'=ef`var' if line == 15
quietly gen allfirstef`var'=ef`var' if efalevel==4
quietly gen ftallef`var'=ef`var' if line == 14
quietly gen ptunderef`var'=ef`var' if line == 22
quietly gen ptprofef`var'=ef`var' if line == 23
quietly gen ptgradef`var'=ef`var' if line == 25
quietly gen ftunderef`var'=ef`var' if line == 8
quietly gen ftprofef`var'=ef`var' if line == 9
quietly gen ftgradef`var'=ef`var' if line == 11
}
**REPLACE ZERO VALUES WITH SMALL NEGATIVE VALUES**
foreach group in totlt totlm totlw aiant aianm aianw asiat asiam ///
asiaw bkaat bkaam bkaaw hispt hispm hispw whitt whitm whitw ///
unknt unknm unknw 2mort 2morm 2morw nralt nralm nralw nhpit ///
nhpim nhpiw minort {
foreach level in ftfirst ptfirst allfirst ftall ptunder ptprof ptgrad ftunder ftprof ftgrad {
quietly bysort unitid `level'ef`group' : gen miss`level'ef`group' = missing(`level'ef`group'[1])
}
}
**COLLAPSE ENROLLMENT VARIABLES TO A SINGLE ROW FOR ALL INSTITUTIONS**
collapse (min) miss* (sum) ft* pt* all*, by(unitid)

foreach group in totlt totlm totlw aiant aianm aianw asiat asiam ///
asiaw bkaat bkaam bkaaw hispt hispm hispw whitt whitm whitw ///
unknt unknm unknw 2mort 2morm 2morw nralt nralm nralw nhpit ///
nhpim nhpiw minort {
foreach level in ftfirst ptfirst allfirst ftall ptunder ptprof ptgrad ftunder ftprof ftgrad {
quietly replace `level'ef`group'=. if miss`level'ef`group'
}
}
    
drop miss* ptprof* ftprof*
save fteenrollment`year', replace

copy https://nces.ed.gov/ipeds/datacenter/data/HD`year'_Data_Stata.zip HD`year'_Data_Stata.zip, replace
unzipfile HD`year'_Data_Stata, replace
quietly insheet using hd`year'_data_stata.csv, clear
quietly sort unitid
quietly local m1=(`year'+1)
quietly save hd_`m1', replace
}
use hd_2019, clear
merge 1:1 unitid using fteenrollment2018, nogen
save hdfallenroll2019, replace
}

In [2]:
quietly {
    use hdfallenroll2019.dta, clear
drop ftall* ftunder* pt* allfirst* ftgrad*
set line 125
ds, alpha varwidth(20)
}

## A. State level segregation measure

### Average percentage of students of other race at school for average student of a given race

In [3]:
quietly {
use hdfallenroll2019, clear
gen wxblackpctall=ftfirstefwhitt *(ftfirstefbkaat / ftfirsteftotlt)
gen wxblackpct4yr=ftfirstefwhitt *(ftfirstefbkaat / ftfirsteftotlt) if iclevel==1
gen wxblackpct2yr=ftfirstefwhitt *(ftfirstefbkaat / ftfirsteftotlt) if iclevel==2

gen wxlatinpctall=ftfirstefwhitt *(ftfirstefhispt / ftfirsteftotlt)
gen wxlatinpct4yr=ftfirstefwhitt *(ftfirstefhispt / ftfirsteftotlt) if iclevel==1
gen wxlatinpct2yr=ftfirstefwhitt *(ftfirstefhispt / ftfirsteftotlt) if iclevel==2

foreach var in ftfirstefbkaat ftfirstefhispt ftfirstefwhitt {
    gen `var'4yr =`var' if iclevel==1
    gen `var'2yr =`var' if iclevel==2
    rename `var' `var'all
}

collapse (sum) ftfirstefbkaat* ftfirstefhispt* ftfirstefwhitt* wx*, by(stabbr)

foreach level in all 4yr 2yr {
    gen wtblk`level'=100*(1/ftfirstefwhitt`level')*wxblackpct`level'
    gen wtltn`level'=100*(1/ftfirstefwhitt`level')*wxlatinpct`level'
    }

foreach var in wtblk wtltn {
    gen `var'dif=`var'4yr - `var'2yr
    }
drop if stabbr=="PR" | stabbr=="GU" | stabbr=="VI" | stabbr=="PW" | stabbr=="AS" | stabbr=="MH" | ///
    stabbr=="MP" | stabbr=="FM"
}

In [4]:
quietly set line 200

quietly sort wtblkdif
display "AVERAGE PERCENTAGE OF STUDENTS WHO ARE BLACK AT SCHOOLS OF AVERAGE WHITE STUDENTS"
display "NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS"
list stabbr wtblk*

quietly sort wtltndif
display "AVERAGE PERCENTAGE OF STUDENTS WHO ARE LATINX AT SCHOOLS OF AVERAGE WHITE STUDENTS"
display "NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS"
list stabbr wtltn*




AVERAGE PERCENTAGE OF STUDENTS WHO ARE BLACK AT SCHOOLS OF AVERAGE WHITE STUDENTS

NEGATIVE DIF = MORE SEGREGATION AT 4 YEARS


     +-----------------------------------------------------+
     | stabbr   wtblkall   wtblk4yr   wtblk2yr    wtblkdif |
     |-----------------------------------------------------|
  1. |     DC   8.160051   8.027805   50.49407   -42.46627 |
  2. |     MS   29.23646    18.8968    37.6819    -18.7851 |
  3. |     LA   22.06199    17.3809   35.27783   -17.89693 |
  4. |     NV   6.262385   5.546621   20.06127   -14.51465 |
  5. |     DE   12.76821    12.2592   26.31579   -14.05659 |
     |-----------------------------------------------------|
  6. |     GA   20.19677   18.37978   32.19826   -13.81849 |
  7. |     FL   12.94791   11.31427   23.14557    -11.8313 |
  8. |     AL      15.81   12.77365   22.01055   -9.236896 |
  9. |     SC   15.19185   12.33119   21.16738   -8.836188 |
 10. |     MN   6.650834   4.586222   11.85947   -7.273243 |
     |---------